## Import necessary modules

In [1]:
import time
import datetime
import pandas as pd
import numpy as np
import datetime as dt
from collections import OrderedDict
from copy import copy
import warnings
import matplotlib.pyplot as plt
%matplotlib inline

## Filepath management

In [2]:
project_dir = r'/Users/hudson/Code/marketModel/'
price_path = project_dir + r'stock_price_data/'

## "BarChart" api access

In [3]:
apikey = 'a207db3b2e61eac30ed9b9cd18b2e0d0'

def construct_barChart_url(sym, start_date, freq='minutes', interval='1', api_key=apikey):
    '''Function to construct barchart api url
    Get the barChart url for
    * sym -- the ticker symbol
    * start_date -- the earliest time in format yyyymmddhhmmss
    * freq -- ['days', 'hours', 'minutes', 'seconds']
    * api_key the barChart api key
    '''
    url = 'http://marketdata.websol.barchart.com/getHistory.csv?' +\
            'key={}&symbol={}&type={}&startDate={}'\
    .format(api_key, sym, freq, start_date)

    return url

In [4]:
api_test_url = construct_barChart_url('GOOG', '20170701000000', 'minutes', apikey)
goog = pd.read_csv(api_test_url, parse_dates=['timestamp'])

In [5]:
print goog.head()
print goog.describe()

  symbol           timestamp  tradingDay    open    high       low     close  \
0   GOOG 2017-06-06 13:30:00  2017-06-06  983.16  983.47  981.0200  983.4700   
1   GOOG 2017-06-06 13:31:00  2017-06-06  984.27  985.59  983.0842  983.6800   
2   GOOG 2017-06-06 13:32:00  2017-06-06  984.45  984.71  982.5900  983.3083   
3   GOOG 2017-06-06 13:33:00  2017-06-06  983.73  984.32  982.9550  984.3200   
4   GOOG 2017-06-06 13:34:00  2017-06-06  984.32  984.53  983.9740  983.9740   

   volume  
0   38054  
1    6259  
2    5213  
3    7581  
4    3415  
               open          high           low         close         volume
count  23940.000000  23940.000000  23940.000000  23940.000000   23940.000000
mean     937.999755    938.245102    937.742463    937.989021    2801.018212
std       20.727614     20.713608     20.740116     20.724061    8160.877618
min      894.860000    895.740000    894.790000    895.000000     100.000000
25%      921.840000    922.050000    921.610000    921.849750 

## Now start pulling in data for our sample stock symbols

In [6]:
symbols = pd.read_csv(project_dir + 'data/stock_data/symbols.csv')

In [7]:
# Pull data for all the test symbols
days_prior_to_now = 30
current = datetime.datetime.now()
starttime = (current - datetime.timedelta(days=days_prior_to_now)).strftime('%Y%m%d%H%M00')
print "stock data start time: " + starttime
prices = {}

for symbol in symbols.ticker_symbol:
    # Construct the appropriate URL
    url = construct_barChart_url(symbol, starttime, 'minutes', apikey)
    print(url)
    
    try:
        # Read the data from the url
        data = pd.read_csv(url, parse_dates=['timestamp']).set_index('timestamp')

        # Drop the symbol and trading day columns
        data = data.drop(['symbol','tradingDay'], axis=1)

        # Convert the times to eastern time zone
        data.index = data.index.tz_localize('utc').tz_convert('US/Eastern')

        # Add data to prices dictionary
        prices[symbol] = data
    except:
        print "Failed to load data for " + symbol
        continue

stock data start time: 20170807160400
http://marketdata.websol.barchart.com/getHistory.csv?key=a207db3b2e61eac30ed9b9cd18b2e0d0&symbol=ARDM&type=minutes&startDate=20170807160400
http://marketdata.websol.barchart.com/getHistory.csv?key=a207db3b2e61eac30ed9b9cd18b2e0d0&symbol=MICR&type=minutes&startDate=20170807160400
http://marketdata.websol.barchart.com/getHistory.csv?key=a207db3b2e61eac30ed9b9cd18b2e0d0&symbol=AVIR&type=minutes&startDate=20170807160400
http://marketdata.websol.barchart.com/getHistory.csv?key=a207db3b2e61eac30ed9b9cd18b2e0d0&symbol=SEAC&type=minutes&startDate=20170807160400
http://marketdata.websol.barchart.com/getHistory.csv?key=a207db3b2e61eac30ed9b9cd18b2e0d0&symbol=CPST&type=minutes&startDate=20170807160400
http://marketdata.websol.barchart.com/getHistory.csv?key=a207db3b2e61eac30ed9b9cd18b2e0d0&symbol=GBR&type=minutes&startDate=20170807160400
http://marketdata.websol.barchart.com/getHistory.csv?key=a207db3b2e61eac30ed9b9cd18b2e0d0&symbol=DTRM&type=minutes&startDat

In [12]:
# Concatenate all of the stock data into a multiIndex dataframe
stock_data = pd.concat(prices.values(), keys=prices.keys())
stock_data.index.set_names(['ticker', 'timestamp'], inplace=True)
stock_data

open    high     low   close  volume
ticker timestamp                                                        
ASTC   2017-08-08 10:47:00-04:00  0.7311  0.7311  0.7311  0.7311    1000
       2017-08-08 10:57:00-04:00  0.7500  0.7500  0.7500  0.7500     300
       2017-08-08 11:04:00-04:00  0.7600  0.7600  0.7600  0.7600     100
       2017-08-08 11:33:00-04:00  0.7301  0.7301  0.7301  0.7301     100
       2017-08-08 11:34:00-04:00  0.7400  0.7401  0.7250  0.7250   18677
       2017-08-08 11:41:00-04:00  0.7400  0.7400  0.7400  0.7400     100
       2017-08-08 11:49:00-04:00  0.7323  0.7488  0.7301  0.7488    5177
       2017-08-08 12:21:00-04:00  0.7698  0.7698  0.7698  0.7698    4000
       2017-08-08 12:40:00-04:00  0.7251  0.7251  0.7251  0.7251    3250
       2017-08-08 12:48:00-04:00  0.7698  0.7698  0.7698  0.7698    1000
       2017-08-08 13:06:00-04:00  0.7699  0.7699  0.7699  0.7699     100
       2017-08-08 13:30:00-04:00  0.7512  0.7512  0.7512  0.7512    1750
       2017-08-08 13:40:00-04:00  0.7699  0.7699  0.7699  0.7699     100
       2017-08-08 13:45:00-04:00  0.8000  0.8000  0.8000  0.8000     100
       2017-08-08 13:58:00-04:00  0.7512  0.7512  0.7512  0.7512     800
       2017-08-08 14:03:00-04:00  0.7500  0.7504  0.7500  0.7504    5300
       2017-08-08 14:54:00-04:00  0.7900  0.7900  0.7900  0.7900     100
       2017-08-08 15:02:00-04:00  0.7895  0.7900  0.7895  0.7900    4000
       2017-08-08 15:13:00-04:00  0.7501  0.7720  0.7501  0.7720    5949
       2017-08-08 15:22:00-04:00  0.7900  0.7900  0.7900  0.7900    1000
       2017-08-08 15:38:00-04:00  0.7500  0.7500  0.7500  0.7500     100
       2017-08-08 15:41:00-04:00  0.7900  0.7900  0.7900  0.7900     100
       2017-08-09 09:30:00-04:00  0.7900  0.7900  0.7900  0.7900     300
       2017-08-09 09:35:00-04:00  0.7900  0.7900  0.7900  0.7900     300
       2017-08-09 10:03:00-04:00  0.8399  0.8399  0.8399  0.8399     100
       2017-08-09 10:31:00-04:00  0.8200  0.8399  0.8200  0.8399    1686
       2017-08-09 10:32:00-04:00  0.8100  0.8100  0.8100  0.8100     400
       2017-08-09 10:38:00-04:00  0.8162  0.8162  0.8162  0.8162    5000
       2017-08-09 11:01:00-04:00  0.8382  0.8382  0.8382  0.8382    1200
       2017-08-09 11:49:00-04:00  0.8228  0.8228  0.8228  0.8228     500
...                                  ...     ...     ...     ...     ...
MOC    2017-08-31 16:00:00-04:00  2.9000  2.9000  2.9000  2.9000     498
       2017-09-01 10:05:00-04:00  3.0200  3.0200  3.0200  3.0200     100
       2017-09-01 11:01:00-04:00  2.9600  2.9600  2.9600  2.9600     100
       2017-09-01 11:07:00-04:00  2.9700  2.9700  2.9700  2.9700     100
       2017-09-01 12:49:00-04:00  2.9700  2.9700  2.9500  2.9500     400
       2017-09-01 13:04:00-04:00  2.9200  2.9200  2.9200  2.9200     100
       2017-09-01 13:06:00-04:00  2.9481  2.9481  2.9481  2.9481     500
       2017-09-01 13:41:00-04:00  2.9200  2.9200  2.8700  2.8700    2100
       2017-09-01 13:42:00-04:00  2.9100  2.9400  2.8800  2.9011    5100
       2017-09-01 13:43:00-04:00  2.9300  2.9700  2.9300  2.9600    3951
       2017-09-01 14:20:00-04:00  2.9632  2.9632  2.9632  2.9632     494
       2017-09-01 15:17:00-04:00  2.9600  2.9600  2.9600  2.9600     200
       2017-09-01 15:19:00-04:00  2.9600  2.9600  2.9600  2.9600     359
       2017-09-01 15:20:00-04:00  2.9600  2.9600  2.9600  2.9600     100
       2017-09-01 15:37:00-04:00  3.0000  3.0200  3.0000  3.0200    5402
       2017-09-01 15:38:00-04:00  3.0000  3.0000  3.0000  3.0000    8301
       2017-09-01 15:45:00-04:00  3.0200  3.0200  2.9900  2.9900     200
       2017-09-05 09:35:00-04:00  2.8500  2.8500  2.8500  2.8500     100
       2017-09-05 09:45:00-04:00  3.0325  3.0325  3.0325  3.0325     750
       2017-09-05 09:46:00-04:00  3.0000  3.0045  2.8800  2.9161    4701
       2017-09-05 09:47:00-04:00  2.8600  2.8600  2.8600  2.8600     100
       2017-09-05 10:05:00-04:00  2.9600  2.9600  2.9600  2.9600     100
       2017-09-05 10:

## Write to hdf

In [9]:
stock_data.to_hdf(project_dir + 'data/stock_data/raw_stock_data.hdf', 'table')